# Determine the sentiment of customer review
1. Reset the data (optional)
2. Create a LLM prompt for the LLM model to determine the sentiment
3. Score the data in batches
4. Extract the text from the scored Json result

## Generate the prompts (Enriched Zone: customer_review)


In [ ]:
%%bigquery

-- OPTIONAL: Reset all the fields to null
-- If you need to reset you data back to fresh data run the stored procedure: CALL `${project_id}.${bigquery_rideshare_llm_curated_dataset}.sp_reset_demo`();
/*
UPDATE `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer_review` AS customer_review
   SET llm_sentiment_prompt = NULL,
       raw_sentiment_json = NULL,
       review_sentiment = NULL
 WHERE TRUE;
*/

In [ ]:
%%bigquery

-- Create the LLM prompt to determine the sentiment of each review
UPDATE `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer_review` AS customer_review
   SET llm_sentiment_prompt =  CONCAT('For the given review classify the sentiment as Positive, Neutral or Negative.','\n','Review: ',customer_review_text)
 WHERE llm_sentiment_prompt IS NULL;

## Score all items in batches
- Find all records that have not been scored
- Score in a batch (we can do up to 10,000)
- The LLM temperature, max_output_tokens, top_p and top_k parameters have been set (locked for a deterministic value)
- Repeat until done

In [ ]:
from google.cloud import bigquery
import pandas as pd

client = bigquery.Client()

In [ ]:
# Process in batches
batch_size = 100

# Set the parameters so we are more deterministic and less creative/random responses
llm_temperature = 1
llm_max_output_tokens = 50
llm_top_p = 0
llm_top_k = 1

update_sql="""
UPDATE `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer_review` AS customer_review
  SET raw_sentiment_json = child.ml_generate_text_result
  FROM (SELECT *
          FROM ML.GENERATE_TEXT(MODEL`${project_id}.${bigquery_rideshare_llm_enriched_dataset}.cloud_ai_llm_v1`,
              (SELECT trip_id,
                      llm_sentiment_prompt AS prompt
                FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer_review`
                WHERE (raw_sentiment_json IS NULL
                       OR
                       JSON_VALUE(raw_sentiment_json, '$.predictions[0].content') IS NULL
                       )
                  AND customer_review_text IS NOT NULL
                  AND llm_sentiment_prompt IS NOT NULL
                  AND review_sentiment IS NULL
                LIMIT {batch_size}),
              STRUCT(
                {llm_temperature} AS temperature,
                {llm_max_output_tokens} AS max_output_tokens,
                {llm_top_p} AS top_p,
                {llm_top_k} AS top_k
                ))
  ) AS child
WHERE customer_review.trip_id = child.trip_id;
  """.format(batch_size = batch_size,
             llm_temperature = llm_temperature,
             llm_max_output_tokens = llm_max_output_tokens,
             llm_top_p = llm_top_p,
             llm_top_k = llm_top_k)

print("SQL: {update_sql}".format(update_sql=update_sql))


In [ ]:
# Score while records remain
# Score in groups of batch_size records (we can do up to 10,000 at a time)
import time

done = False
original_record_count = 0
displayed_first_sql = False

while done == False:
  # Get the count of records to score
  sql = """
        SELECT COUNT(*) AS cnt
          FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer_review`
         WHERE (raw_sentiment_json IS NULL
                OR
                JSON_VALUE(raw_sentiment_json, '$.predictions[0].content') IS NULL
                )
           AND customer_review_text IS NOT NULL
           AND llm_sentiment_prompt IS NOT NULL
           AND review_sentiment IS NULL
        """

  df_record_count = client.query(sql).to_dataframe()
  cnt = df_record_count['cnt'].head(1).item()
  if displayed_first_sql == False:
    original_record_count = cnt
    displayed_first_sql = True

  print("Remaining records to process: ", cnt, " out of", original_record_count, " batch_size: ", batch_size)

  if cnt == 0:
    done = True
  else:
    # https://github.com/googleapis/python-bigquery/tree/master/samples
    job_config = bigquery.QueryJobConfig(priority=bigquery.QueryPriority.INTERACTIVE)
    query_job = client.query(update_sql, job_config=job_config)

    # Check on the progress by getting the job's updated state.
    query_job = client.get_job(
        query_job.job_id, location=query_job.location
    )
    print("Job {} is currently in state {}".format(query_job.job_id, query_job.state))

    while query_job.state != "DONE":
      time.sleep(5)
      query_job = client.get_job(
          query_job.job_id, location=query_job.location
          )
      print("Job {} is currently in state {}".format(query_job.job_id, query_job.state))


## Parse the LLM results to get a rating
- Rating should be Positive, Neutral or Negative

In [ ]:
%%bigquery

SELECT trip_id, llm_sentiment_prompt, raw_sentiment_json
  FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer_review`
 WHERE llm_sentiment_prompt IS NOT NULL
   AND raw_sentiment_json IS NOT NULL
   AND review_sentiment IS NULL
LIMIT 20;

In [ ]:
%%bigquery

UPDATE `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer_review` AS customer_review
   SET review_sentiment = JSON_VALUE(raw_sentiment_json, '$.predictions[0].content')
 WHERE review_sentiment IS NULL
   AND raw_sentiment_json IS NOT NULL
;

In [ ]:
%%bigquery

SELECT trip_id, llm_sentiment_prompt, raw_sentiment_json, review_sentiment
  FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer_review`
 WHERE review_sentiment IS NOT NULL
   AND raw_sentiment_json IS NOT NULL
LIMIT 20;

In [ ]:
%%bigquery

SELECT review_sentiment, COUNT(*) AS cnt
  FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer_review`
 WHERE review_sentiment IS NOT NULL
   AND raw_sentiment_json IS NOT NULL
GROUP BY 1;
